# 【問題1】機械翻訳の実行とコードリーディング
以下のサンプルコードは、短い英語からフランス語への変換を行うものです。これを動かしてください。


keras/lstm_seq2seq.py at master · rstudio/keras


その上でこのサンプルコードの各部分がどういった役割かを読み取り、まとめてください。以下のようにどこからどこの行が何をしているかを記述してください。

In [4]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'eng-fra.txt'

# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

# データを一行毎引っ張り出してきて、textをcharにユニークに格納している
lines = open(data_path).read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

            
#　単語のソート、数、長さを取得
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

# 表示
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

# 単語とインデックスを辞書形式に変更
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

# 文章の数、 最大の単語数、　ユニークな文字の数で初期化
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')


# エンコーダ、デコーダに、何個目の文章か、　シーケンスの何個目か、　文字のinndex番号を代入
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        
        # スタートを除く、上記情報を格納
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.


# 入力(バッチ数、　単語のトークン化した文字の数)
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))

# 出力は(バッチ、256), また中間の重みを出力する。
encoder = LSTM(latent_dim, return_state=True)

# 出力は(256, hの重み, cの重み)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))

# 上のエンコーダーのLSTMの重みで初期化してる
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)

# 出力層はsoftmax
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


# pytorchの書き方、　encoderとdecoderを同じに学習させてその後に出力層を通す
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')

# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models

# Next: 推論モード（サンプリング）.
# 以下はその手順である。
# 1) 入力を符号化し、デコーダの初期状態を取得する
# 2) この初期状態でデコーダの1ステップを実行する
# 2) この初期状態でデコーダの1ステップを実行する # そして "start of sequence" トークンをターゲットにする
# 出力は次のターゲットトークンとなる
# 3) 3) 現在のターゲット・トークンと現在の状態で繰り返す

# サンプリングモデルの定義
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)


# 出力された値を辞書形にして保存する。
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # 文章の終わりの空白が来るまでエンコーダー層からデコーダー層に値を上げる
    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # 単語で回してる
        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # 1文章終わったかの判断
        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # 次の文章のために初期化
        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


# 100回出力を学ばさせる
for seq_index in range(100):
    # Take one sequence (part of the training test)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)


Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 91
Max sequence length for inputs: 16
Max sequence length for outputs: 59
Epoch 1/100
125/125 [==============================] - ETA: 4:20 - loss: 1.495 - ETA: 14s - loss: 1.498 - ETA: 14s - loss: 1.49 - ETA: 13s - loss: 1.40 - ETA: 13s - loss: 1.34 - ETA: 13s - loss: 1.30 - ETA: 13s - loss: 1.26 - ETA: 13s - loss: 1.23 - ETA: 13s - loss: 1.21 - ETA: 13s - loss: 1.19 - ETA: 13s - loss: 1.18 - ETA: 13s - loss: 1.18 - ETA: 12s - loss: 1.17 - ETA: 12s - loss: 1.15 - ETA: 12s - loss: 1.15 - ETA: 12s - loss: 1.14 - ETA: 12s - loss: 1.14 - ETA: 12s - loss: 1.13 - ETA: 12s - loss: 1.12 - ETA: 12s - loss: 1.12 - ETA: 12s - loss: 1.12 - ETA: 11s - loss: 1.11 - ETA: 11s - loss: 1.11 - ETA: 11s - loss: 1.11 - ETA: 11s - loss: 1.11 - ETA: 11s - loss: 1.10 - ETA: 11s - loss: 1.10 - ETA: 11s - loss: 1.10 - ETA: 11s - loss: 1.10 - ETA: 11s - loss: 1.09 - ETA: 10s - loss: 1.09 - ETA: 10s - loss: 1.09 - ETA: 10s 

# 【問題2】イメージキャプショニングの学習済みモデルの実行
上記実装において 5. Test the model の項目を実行してください。また、自身で用意した画像に対しても文章を生成してください。これらに対してどういった文章が出力されたかを記録して提出してください。


データセットからの学習は行わず、学習済みの重みをダウンロードして利用します。


注意点として、デフォルトで設定されている重みのファイル名と、ダウンロードできる重みのファイル名は異なっています。ここは書き換える必要があります。

参考
[http://cedro3.com/ai/pytorch-image-captioning/](http://cedro3.com/ai/pytorch-image-captioning/)

![Imgur](https://i.imgur.com/0PSG4CQh.png)

# 【問題3】Kerasで動かしたい場合はどうするかを調査
PyTorchによる実装を動かしましたが、何らかの理由からKerasで動かしたい状況が考えられます。どういった手順を踏むことになるか調査し、できるだけ詳しく説明してください。


特に今回はPyTorchのための学習済みの重みをKerasで使えるようにしたいので、その点については必ず触れてください。

MMdnnの使用  

- srcFrameworkコマンド  
    変換元フレームワークを入力(今回であればPytorch)  
    
- inputWeightコマンド  
    重みのpathを指定することで、重みを変換できる(今回であればvocab.pki)

- inputNetworkコマンド  
    モデルを入力する(今回であればdecoder-5-3000.pki, encoder-5-3000.pki)  
 
- dstFrameworkコマンド  
    変換したいライブラリを入力(今回であれば、keras)  
    
- dstFrameworkコマンド  
    変換後の拡張子指定(今回であれば.h5)


双方のフレームワークがインストールされていることが条件